### DMD Ingredient->Product Lookup

Get a list of all VMPs (Virtual Medicinal Products) and AMPs (Actual Medicinal Products) from a given list of ingredients (VTMs = Virtual Therapeutic Moieties)

In [4]:
# set name of codelist for exporting file
codelist_name="covid_meds"


# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Tocilizumab', 'Sarilumab']




from ebmdatalab import bq
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



## Select products from DMD containing chosen ingredients

In [2]:
# convert list to tuple for use in SQL query
names_tuple = tuple(names)
if len(names_tuple)==1:
    # remove comma if only one item
    names_tuple = str(names_tuple).replace(",","")

sql = f'''
SELECT "vmp" AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.ddd on vmp.id=ddd.vpid

UNION ALL

SELECT "amp" AS type, amp.id, bnf_code, descr as nm, ing.nm as ingredient, ddd.ddd
FROM dmd.amp
INNER JOIN dmd.vpi AS vpi ON amp.vmp=vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing AND ing.nm IN {names_tuple}
LEFT JOIN dmd.ddd on amp.vmp=ddd.vpid

ORDER BY type, nm  '''

meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'meds_{codelist_name}.csv'))

meds.count()

C:\Users\hcurtis\Anaconda3\envs\antibiotics-rct\lib\site-packages\pandas_gbq\gbq.py:547: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  df = rows_iter.to_dataframe(


type          36
id            36
bnf_code      18
nm            36
ingredient    36
ddd           18
dtype: int64

#### Check if any ingredients were not found

In [3]:
ings = list(meds.ingredient.drop_duplicates())
names.sort()
ings.sort()
print("Selected ingredients:", names)
print("Ingredients found:", ings)

missing_ings = [i for i in names if i not in ings]
print("Ingredients NOT found:", missing_ings)

Selected ingredients: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients found: ['Casirivimab', 'Imdevimab', 'Remdesivir', 'Sarilumab', 'Tocilizumab']
Ingredients NOT found: []


In [8]:
# check how many products have DDDs
print(meds.groupby(["type"])[["ddd", "id"]].count())

      ddd  id
type         
amp     9  20
vmp     9  16


In [9]:
meds

,type,id,bnf_code,nm,ingredient,ddd,ddd_flag
0,amp,39652711000001109,None,Casirivimab (REGN10933) 120mg/ml solution for infusion 11.1ml vials (University of Oxford),Casirivimab,NaN,0
1,amp,40063011000001100,None,Casirivimab 120mg/ml solution for infusion 11.1ml vials (Roche Products Ltd),Casirivimab,NaN,0
2,amp,40025211000001101,None,Casirivimab 120mg/ml solution for infusion 2.5ml vials (Roche Products Ltd),Casirivimab,NaN,0
3,amp,39653411000001107,None,Imdevimab (REGN10987) 120mg/ml solution for infusion 11.1ml vials (University of Oxford),Imdevimab,NaN,0
4,amp,40063211000001105,None,Imdevimab 120mg/ml solution for infusion 11.1ml vials (Roche Products Ltd),Imdevimab,NaN,0
5,amp,40025511000001103,None,Imdevimab 120mg/ml solution for infusion 2.5ml vials (Roche Products Ltd),Imdevimab,NaN,0
6,amp,34735911000001107,1001030ADBBACAC,Kevzara 150mg/1.14ml solution for injection pre-filled pens (Sanofi Genzyme),Sarilumab,14.3,1
7,amp,34735311000001106,1001030ADBBAAAA,Kevzara 150mg/1.14ml solution for injection pre-filled syringes (Sanofi Genzyme),Sarilumab,14.3,1
8,amp,34736211000001109,1001030ADBBADAD,Kevzara 200mg/1.14ml solution for injection pre-filled pens (Sanofi Genzyme),Sarilumab,14.3,1
9,amp,34735611000001101,1001030ADBBABAB,Kevzara 200mg/1.14ml solution for injection pre-filled syringes (Sanofi Genzyme),Sarilumab,14.3,1
